In [1]:
import visual_behavior_glm.src.GLM_params as glm_params
import visual_behavior_glm.src.GLM_analysis_tools as gat
import visual_behavior_glm.src.GLM_visualization_tools as gvt
from visual_behavior_glm.src.glm import GLM
import matplotlib.pyplot as plt
import visual_behavior.data_access.loading as loading
import visual_behavior.database as db
import plotly.express as px

import pandas as pd
import numpy as np
import os

import seaborn as sns

import visual_behavior.plotting as vbp

import plotly.express as px
from sklearn.cluster import KMeans
import umap

/home/dougo/Code/AllenSDK/allensdk/brain_observatory/behavior/behavior_ophys_api/behavior_ophys_nwb_api.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
%matplotlib notebook
%widescreen

applied a custom magic command to make full use of screen width
will only work if command is defined locally
replace with the following to replicate functionality: 
	from IPython.core.display import display, HTML
	display(HTML("<style>.container { width:100% !important; }</style>")


In [19]:
etable = loading.get_filtered_ophys_experiment_table()

In [20]:
len(etable)

1418

# Gather/organize data

## load the results summary for a single GLM version from mongo

In [31]:
glm_version = '6_L2_optimize_by_session'
full_summary = gat.retrieve_results(search_dict = {'glm_version': glm_version}, results_type='full')
full_summary.sample(10)

,_id,cell_specimen_id,Full__avg_cv_var_train,Full__avg_cv_var_test,Full__avg_cv_var_test_full_comparison,Full__avg_cv_adjvar_train,Full__avg_cv_adjvar_test,Full__avg_cv_adjvar_test_full_comparison,Full__absolute_change_from_full,Full__adj_dropout,Full__dropout,intercept__avg_cv_var_train,intercept__avg_cv_var_test,intercept__avg_cv_var_test_full_comparison,intercept__avg_cv_adjvar_train,intercept__avg_cv_adjvar_test,intercept__avg_cv_adjvar_test_full_comparison,intercept__absolute_change_from_full,intercept__adj_dropout,intercept__dropout,time__avg_cv_var_train,time__avg_cv_var_test,time__avg_cv_var_test_full_comparison,time__avg_cv_adjvar_train,time__avg_cv_adjvar_test,time__avg_cv_adjvar_test_full_comparison,time__absolute_change_from_full,time__adj_dropout,time__dropout,pre_licks__avg_cv_var_train,pre_licks__avg_cv_var_test,pre_licks__avg_cv_var_test_full_comparison,pre_licks__avg_cv_adjvar_train,pre_licks__avg_cv_adjvar_test,pre_licks__avg_cv_adjvar_test_full_comparison,pre_licks__absolute_change_from_full,pre_licks__adj_dropout,pre_licks__dropout,post_licks__avg_cv_var_train,post_licks__avg_cv_var_test,post_licks__avg_cv_var_test_full_comparison,post_licks__avg_cv_adjvar_train,post_licks__avg_cv_adjvar_test,post_licks__avg_cv_adjvar_test_full_comparison,post_licks__absolute_change_from_full,post_licks__adj_dropout,post_licks__dropout,pre_lick_bouts__avg_cv_var_train,pre_lick_bouts__avg_cv_var_test,pre_lick_bouts__avg_cv_var_test_full_comparison,pre_lick_bouts__avg_cv_adjvar_train,pre_lick_bouts__avg_cv_adjvar_test,pre_lick_bouts__avg_cv_adjvar_test_full_comparison,pre_lick_bouts__absolute_change_from_full,pre_lick_bouts__adj_dropout,pre_lick_bouts__dropout,post_lick_bouts__avg_cv_var_train,post_lick_bouts__avg_cv_var_test,post_lick_bouts__avg_cv_var_test_full_comparison,post_lick_bouts__avg_cv_adjvar_train,post_lick_bouts__avg_cv_adjvar_test,post_lick_bouts__avg_cv_adjvar_test_full_comparison,post_lick_bouts__absolute_change_from_full,post_lick_bouts__adj_dropout,post_lick_bouts__dropout,rewards__avg_cv_var_train,rewards__avg_cv_var_test,rewards__avg_cv_var_test_full_comparison,rewards__avg_cv_adjvar_train,rewards__avg_cv_adjvar_test,rewards__avg_cv_adjvar_test_full_comparison,rewards__absolute_change_from_full,rewards__adj_dropout,rewards__dropout,change__avg_cv_var_train,change__avg_cv_var_test,change__avg_cv_var_test_full_comparison,change__avg_cv_adjvar_train,change__avg_cv_adjvar_test,change__avg_cv_adjvar_test_full_comparison,change__absolute_change_from_full,change__adj_dropout,change__dropout,hits__avg_cv_var_train,hits__avg_cv_var_test,hits__avg_cv_var_test_full_comparison,hits__avg_cv_adjvar_train,hits__avg_cv_adjvar_test,hits__avg_cv_adjvar_test_full_comparison,hits__absolute_change_from_full,hits__adj_dropout,hits__dropout,misses__avg_cv_var_train,misses__avg_cv_var_test,misses__avg_cv_var_test_full_comparison,misses__avg_cv_adjvar_train,misses__avg_cv_adjvar_test,misses__avg_cv_adjvar_test_full_comparison,misses__absolute_change_from_full,misses__adj_dropout,misses__dropout,false_alarms__avg_cv_var_train,false_alarms__avg_cv_var_test,false_alarms__avg_cv_var_test_full_comparison,false_alarms__avg_cv_adjvar_train,false_alarms__avg_cv_adjvar_test,false_alarms__avg_cv_adjvar_test_full_comparison,false_alarms__absolute_change_from_full,false_alarms__adj_dropout,false_alarms__dropout,correct_rejects__avg_cv_var_train,correct_rejects__avg_cv_var_test,correct_rejects__avg_cv_var_test_full_comparison,correct_rejects__avg_cv_adjvar_train,correct_rejects__avg_cv_adjvar_test,correct_rejects__avg_cv_adjvar_test_full_comparison,correct_rejects__absolute_change_from_full,correct_rejects__adj_dropout,correct_rejects__dropout,omissions__avg_cv_var_train,omissions__avg_cv_var_test,omissions__avg_cv_var_test_full_comparison,omissions__avg_cv_adjvar_train,omissions__avg_cv_adjvar_test,omissions__avg_cv_adjvar_test_full_comparison,omissions__absolute_change_from_full,omissions__adj_dropout,omissions__dropout,image_expectation__a

In [32]:
len(full_summary['ophys_experiment_id'].unique())

949

In [23]:
glm_version = '6_L2_optimize_by_session'
dropout_summary = gat.retrieve_results(search_dict = {'glm_version': glm_version}, results_type='summary')
len(dropout_summary['ophys_experiment_id'].unique())

824

In [24]:
glm_version = '6_L2_optimize_by_cell'
dropout_summary = gat.retrieve_results(search_dict = {'glm_version': glm_version}, results_type='summary')
len(dropout_summary['ophys_experiment_id'].unique())

560

In [30]:
count = 0
for idx,row in etable.reset_index().iterrows():
    if row['ophys_experiment_id'] not in list(dropout_summary['ophys_experiment_id'].unique()):
        count += 1
        print('oeid {} is missing, count = {}'.format(row['ophys_experiment_id'], count))

oeid 953659745 is missing, count = 1
oeid 953659749 is missing, count = 2
oeid 953659752 is missing, count = 3
oeid 953659743 is missing, count = 4
oeid 958527474 is missing, count = 5
oeid 958527464 is missing, count = 6
oeid 958527471 is missing, count = 7
oeid 958527477 is missing, count = 8
oeid 958527479 is missing, count = 9
oeid 958527481 is missing, count = 10
oeid 958527485 is missing, count = 11
oeid 958527488 is missing, count = 12
oeid 956941844 is missing, count = 13
oeid 956941848 is missing, count = 14
oeid 956941841 is missing, count = 15
oeid 956941846 is missing, count = 16
oeid 957759564 is missing, count = 17
oeid 957759566 is missing, count = 18
oeid 957759574 is missing, count = 19
oeid 957759570 is missing, count = 20
oeid 957759562 is missing, count = 21
oeid 957759568 is missing, count = 22
oeid 957759572 is missing, count = 23
oeid 957759576 is missing, count = 24
oeid 958741226 is missing, count = 25
oeid 958741234 is missing, count = 26
oeid 958741232 is mis

oeid 905955228 is missing, count = 220
oeid 905955230 is missing, count = 221
oeid 905955232 is missing, count = 222
oeid 906910595 is missing, count = 223
oeid 907694950 is missing, count = 224
oeid 907694952 is missing, count = 225
oeid 907694954 is missing, count = 226
oeid 908381694 is missing, count = 227
oeid 908381700 is missing, count = 228
oeid 909177759 is missing, count = 229
oeid 912791343 is missing, count = 230
oeid 912791341 is missing, count = 231
oeid 912791345 is missing, count = 232
oeid 914107600 is missing, count = 233
oeid 914107611 is missing, count = 234
oeid 914107608 is missing, count = 235
oeid 914107592 is missing, count = 236
oeid 914107595 is missing, count = 237
oeid 914580662 is missing, count = 238
oeid 914580670 is missing, count = 239
oeid 914580664 is missing, count = 240
oeid 914580660 is missing, count = 241
oeid 915243101 is missing, count = 242
oeid 915243090 is missing, count = 243
oeid 915243092 is missing, count = 244
oeid 915243099 is missing

oeid 1006562220 is missing, count = 484
oeid 1009495888 is missing, count = 485
oeid 1009495889 is missing, count = 486
oeid 1009495892 is missing, count = 487
oeid 1009495893 is missing, count = 488
oeid 1009495897 is missing, count = 489
oeid 1009495886 is missing, count = 490
oeid 1010092813 is missing, count = 491
oeid 1010092822 is missing, count = 492
oeid 1010092819 is missing, count = 493
oeid 1010092818 is missing, count = 494
oeid 1010092816 is missing, count = 495
oeid 1010092815 is missing, count = 496
oeid 1010557566 is missing, count = 497
oeid 1010557563 is missing, count = 498
oeid 1010557557 is missing, count = 499
oeid 1010557562 is missing, count = 500
oeid 1010557560 is missing, count = 501
oeid 1012165666 is missing, count = 502
oeid 1012165657 is missing, count = 503
oeid 1012165660 is missing, count = 504
oeid 1012165662 is missing, count = 505
oeid 1012165663 is missing, count = 506
oeid 1012771667 is missing, count = 507
oeid 1012771673 is missing, count = 508


oeid 847267630 is missing, count = 706
oeid 848039121 is missing, count = 707
oeid 848039125 is missing, count = 708
oeid 848039123 is missing, count = 709
oeid 848760990 is missing, count = 710
oeid 848760983 is missing, count = 711
oeid 848760985 is missing, count = 712
oeid 848760988 is missing, count = 713
oeid 849233400 is missing, count = 714
oeid 849233404 is missing, count = 715
oeid 849233402 is missing, count = 716
oeid 849233398 is missing, count = 717
oeid 849233390 is missing, count = 718
oeid 849233396 is missing, count = 719
oeid 851085103 is missing, count = 720
oeid 851085105 is missing, count = 721
oeid 851085107 is missing, count = 722
oeid 851085109 is missing, count = 723
oeid 851085092 is missing, count = 724
oeid 851085100 is missing, count = 725
oeid 853362773 is missing, count = 726
oeid 853362771 is missing, count = 727
oeid 853362777 is missing, count = 728
oeid 853362775 is missing, count = 729
oeid 853362765 is missing, count = 730
oeid 889771676 is missing

In [ ]:
s = '{} {} {}'.format(1,2,3)